In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import sklearn.metrics as metrics

In [59]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [60]:
columns = train.columns.values
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [61]:
# Check which columns have missing values
missing = train.isnull().sum().sort_values(ascending=False)
missing

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [62]:
means = [37,29,24]
def fill_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return means[0]
        elif Pclass == 2:
            return means[1]
        else:
            return means[2]
    else:
        return cols[0]

In [64]:
train['Age'] = train[['Age','Pclass']].apply(fill_age,axis=1)
test['Age'] = test[['Age','Pclass']].apply(fill_age,axis=1)

In [67]:
train.drop('Cabin',axis='columns',inplace=True) 
test.drop('Cabin',axis='columns',inplace=True)
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [77]:
# Create dummies for the categorical variables
sex = pd.get_dummies(train['Sex'],drop_first=True)  
embark = pd.get_dummies(train['Embarked'],drop_first=True)
#drop the categorical variables
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

train = pd.concat([train,sex,embark],axis=1)

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1),train['Survived'], train_size=0.8) 

In [79]:
#And now, let's train our model
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()

#Great, now, let's predict the survived passengers on the test dataset and assign them to predictions variable:
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

/home/gatik/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [86]:
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

Accuracy: 0.7640449438202247
